In [110]:
# 导入需要的库
import pandas as pd
import numpy as np
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.style.use('ggplot')



def ret_df(w):
    path = 'D:\\Quant_Code\\data\\index\\weekly_index.csv'
#     path = 'D:\\Quant_Code\\data\\index\\weekly_sw_3.csv'
    df = pd.read_csv(path)
    df_list =  list()
    codes = df['code'].unique().tolist()
    for i in codes:
        df_t = df[df['code'] == i]
        df_t.reset_index(drop=True,inplace=True)
        df_t['ret'] =(df_t['close_last']/df_t['close_last'].shift(w)-1)         
        df_t= df_t.iloc[w:].fillna(0)     
        df_list.append(np.median(df_t['ret']))
 
    res = sorted(dict(zip(codes,df_list)).items(),key=lambda x:x[1],reverse=True)
    codes_sorted = list()
    for i in res:
        codes_sorted.append(i[0])
          
    return codes_sorted

def get_ETF_filt(path='D:\\Quant_Code\\data\\ETF\\ETF_filt.csv'):  
    df = pd.read_csv(path,encoding='utf-8')
    df['fee'] = df['fee'].apply(lambda x: format_str(x))
    df['fee'] = df['fee'].astype('float')
    return df

   
def format_str(se):
    return re.sub(r'[\%]', "", str(se))
def format_str_code(se):
    return re.sub(r'[\D]', "", str(se))

def get_ETF_codes():   
    codes = ret_df(12)[0:66]
    df = get_ETF_filt()
    codes_list = list()
    names_list = list()
    for code in codes:
        df_t = df[df['index_code']==code]
        df_t.sort_values(by=['fee'],ascending=True,inplace=True)
        if df_t['name'].tolist()[0:2]:
            codes_list.append(df_t['name'].tolist()[0])
            names_list.append(df_t['code'].tolist()[0])
    [format_str_code(i) for i in names_list]
    return [format_str_code(i) for i in names_list],codes_list

def get_weekly_ETF_data(path='D:\\Quant_Code\\data\\ETF\\weekly_ETF.csv'):
    df_list =list()
    df = pd.read_csv(path)
    df['code'] = df['code'].apply(lambda x: format_str_code(x))
    cols =['date','code','close_last']
    names,codes = get_ETF_codes()
    
    df_t = df[cols].copy()
    for code in names:        
        df_s = df_t[df_t['code']==code].copy()
        if df_s.shape[0] >= 52:        
            df_s = df_s.iloc[-52:].copy()
            df_s.set_index(['date','code'],inplace=True)
            df_list.append(df_s)
    df_res =pd.concat(df_list,axis=0)
    cols = [''.join(col) for col in df_res.columns.values] #把双层索引连接成字符串
    df_res.columns = [format_str_code(i) for i in cols] #保留双层索引数字部分
    
    return df_res.unstack()


df = get_weekly_ETF_data()


df.columns = [i[1] for  i in df.columns] #把双层索引转为单层索引

df


ipykernel_launcher:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,159807,159906,159928,159929,159949,159952,159962,159966,159967,159973,...,512670,512690,512710,512770,515000,515030,515200,515210,515580,515790
date,,,,,,,,,,,,,,,,,,,,,
2020-12-27,1.623,1.563,5.215,2.371,1.247,1.726,1.498,0.646,0.658,1.859,...,2.081,2.689,1.831,2.130,1.813,1.664,1.898,1.233,1.651,1.136
2021-01-03,1.739,1.651,5.647,2.479,1.328,1.829,1.544,0.668,0.702,2.036,...,2.263,2.876,1.995,2.328,1.930,1.802,2.049,1.272,1.738,1.219
2021-01-10,1.714,1.619,5.286,2.399,1.314,1.794,1.499,0.656,0.698,1.971,...,2.079,2.650,1.860,2.284,1.940,1.714,2.014,1.256,1.729,1.167
2021-01-17,1.818,1.718,5.315,2.620,1.433,1.951,1.553,0.703,0.777,2.096,...,2.163,2.652,1.949,2.477,2.023,1.839,2.136,1.274,1.859,1.287
2021-01-24,1.671,1.638,5.324,2.450,1.330,1.819,1.557,0.664,0.721,2.008,...,1.942,2.678,1.737,2.303,1.858,1.688,1.992,1.229,1.713,1.224
2021-01-31,1.675,1.706,5.571,2.570,1.377,1.855,1.603,0.684,0.739,2.005,...,1.822,2.833,1.641,2.348,1.833,1.673,1.996,1.238,1.698,1.202
2021-02-07,1.779,1.801,6.025,2.748,1.476,1.981,1.699,0.724,0.789,2.166,...,1.941,3.046,1.748,2.529,1.934,1.774,2.135,1.259,1.812,1.279
2021-02-14,1.717,1.710,5.964,2.657,1.406,1.905,1.752,0.707,0.746,2.133,...,1.915,2.919,1.727,2.431,1.890,1.712,2.068,1.326,1.774,1.244
2021-02-21,1.580,1.580,5.210,2.393,1.241,1.693,1.583,0.637,0.644,1.923,...,1.873,2.433,1.677,2.178,1.771,1.520,1.893,1.337,1.659,1.141


In [112]:
29 / 2

14.5

In [112]:
import json 
path = 'D:\E-BOOK\ETF_1.csv'
df = pd.read_csv(path)
index_dic= dict(
    zip(df['跟踪指数简称'].values.tolist(),df['跟踪指数代码'].values.tolist())
)

with open('D:\E-BOOK\指数信息.txt', 'w') as f:
    json_str = json.dumps(index_dic,indent=0,ensure_ascii=False)
    f.write(json_str)
    f.write('\n')

In [104]:
import json 
path = 'D:\E-BOOK\ETF_1.csv'
df = pd.read_csv(path)
index_dic= dict(
    zip(df['证券名称'].values.tolist(),df['跟踪指数代码'].values.tolist())
)

with open('D:\E-BOOK\ETF2index.txt', 'w') as f:
    json_str = json.dumps(index_dic,indent=0,ensure_ascii=False)
    f.write(json_str)
    f.write('\n')


In [17]:
import json
import pprint

pp = pprint.PrettyPrinter(indent=4)
def read_file(path):
    with open(path,'r') as f:
        file_str = f.read()
        dic = json.loads(file_str)
    return dic 

path_index = 'D:\E-BOOK\ETF2index.txt'
path_code = 'D:\E-BOOK\ETF2code.txt'
index_info = 'D:\E-BOOK\指数信息.txt'

# pp.pprint(read_file(index_info))   
index_code = read_file(index_info).values()
index_code
import re 
pattern = re.compile(r'.+?\.SH$')   # 查找.SH 上交所指数
res_sh =[]
for i in index_code:
    find = pattern.findall(i)
    if find:
        [res_sh.append(item) for item in find]
    
res_sh

['000852.SH',
 '000300.SH',
 '000016.SH',
 '000905.SH',
 '000688.SH',
 '000010.SH',
 '000001.SH',
 '000906.SH',
 '000903.SH',
 '000044.SH',
 '000009.SH',
 '000046.SH',
 '000819.SH',
 '000913.SH',
 '000932.SH',
 '000018.SH',
 '000991.SH',
 '000036.SH',
 '000993.SH',
 '000933.SH',
 '000928.SH',
 '000038.SH',
 '000992.SH',
 '000858.SH',
 '950105.SH',
 '000037.SH',
 '000934.SH',
 '000987.SH',
 '000986.SH',
 '000989.SH',
 '000974.SH',
 '000914.SH',
 '000841.SH',
 '000935.SH',
 '000015.SH',
 '000922.SH',
 '000068.SH',
 '000827.SH',
 '000069.SH',
 '950041.SH',
 '000806.SH',
 '000066.SH',
 '000814.SH',
 '000823.SH',
 '000979.SH',
 '000063.SH',
 '000821.SH',
 '000065.SH',
 '000808.SH',
 '000901.SH',
 '000042.SH',
 '000802.SH',
 '000805.SH',
 '000021.SH',
 '000961.SH',
 '000048.SH',
 '000056.SH',
 '000998.SH',
 '000064.SH',
 '000963.SH',
 '000029.SH',
 '000043.SH',
 '000925.SH',
 '000982.SH',
 'H50040.SH',
 '000971.SH',
 '000984.SH',
 'H50069.SH']

In [18]:
pattern = re.compile(r'.+?\.SZ$')   # 查找.SZ 深交所指数
res_sz =[]
for i in index_code:
    find = pattern.findall(i)
    if find:
        [res_sz.append(item) for item in find]
    
res_sz

['399006.SZ',
 '399330.SZ',
 '399005.SZ',
 '399606.SZ',
 '399001.SZ',
 '399007.SZ',
 '399008.SZ',
 '399624.SZ',
 '399101.SZ',
 '399975.SZ',
 '399986.SZ',
 '399989.SZ',
 '399998.SZ',
 '399396.SZ',
 '399440.SZ',
 '399393.SZ',
 '399965.SZ',
 '980017.SZ',
 '399967.SZ',
 '399987.SZ',
 '399976.SZ',
 '399417.SZ',
 '399808.SZ',
 '399973.SZ',
 '399437.SZ',
 '399971.SZ',
 '399324.SZ',
 '399997.SZ',
 '399995.SZ',
 '399441.SZ',
 '399966.SZ',
 '399699.SZ',
 '399395.SZ',
 '980001.SZ',
 '399806.SZ',
 '399707.SZ',
 '399809.SZ',
 '399291.SZ',
 '399364.SZ',
 '399550.SZ',
 '399368.SZ',
 '399394.SZ',
 '399974.SZ',
 '399970.SZ',
 '399610.SZ',
 '399804.SZ',
 '399993.SZ',
 '399802.SZ',
 '399362.SZ',
 '399807.SZ',
 '399803.SZ',
 '399811.SZ',
 '399992.SZ',
 '399991.SZ',
 '399422.SZ',
 '399293.SZ',
 '399377.SZ',
 '399348.SZ',
 '399673.SZ',
 '399296.SZ',
 '399295.SZ',
 '399326.SZ',
 '399702.SZ',
 '399990.SZ',
 '399701.SZ']

In [19]:
pattern = re.compile(r'.+?\.CSI$')   # 查找.CSI 中证指数
res_csi =[]
for i in index_code:
    find = pattern.findall(i)
    if find:
        [res_csi.append(item) for item in find]
    
res_csi

['H30184.CSI',
 'H30035.CSI',
 'H30165.CSI',
 '930606.CSI',
 '931160.CSI',
 '930697.CSI',
 '930708.CSI',
 '930851.CSI',
 '930614.CSI',
 '931151.CSI',
 '931079.CSI',
 '990001.CSI',
 '000815.CSI',
 '931066.CSI',
 '931152.CSI',
 '931087.CSI',
 '930726.CSI',
 '930743.CSI',
 '930652.CSI',
 '000813.CSI',
 '931139.CSI',
 '930598.CSI',
 '000949.CSI',
 '931380.CSI',
 '930901.CSI',
 '931461.CSI',
 '930713.CSI',
 '930651.CSI',
 'H00813.CSI',
 'H30202.CSI',
 '930601.CSI',
 '930721.CSI',
 '931186.CSI',
 '930707.CSI',
 '000171.CSI',
 '931406.CSI',
 '931187.CSI',
 '930850.CSI',
 '931071.CSI',
 '930648.CSI',
 '930781.CSI',
 '931159.CSI',
 '930902.CSI',
 '000861.CSI',
 '931140.CSI',
 'H30178.CSI',
 '931165.CSI',
 '000811.CSI',
 'H30318.CSI',
 'H30089.CSI',
 '931068.CSI',
 '950096.CSI',
 '931167.CSI',
 '000860.CSI',
 '931372.CSI',
 '000859.CSI',
 '931381.CSI',
 '931166.CSI',
 '931268.CSI',
 '931141.CSI',
 '930875.CSI',
 '931163.CSI',
 '930719.CSI',
 '930758.CSI',
 '931033.CSI',
 '930620.CSI',
 '930703.C

In [31]:
import tushare as ts
pro = ts.pro_api()

df = pro.index_basic(market='SSE')
s1 = set(df['ts_code'].tolist())
s2 = set(res_sh)
print(len(s1),len(s2))
print(len(s1 & s2)) 
index_sh = list(s1 & s2)

595 68
57


In [32]:
df = pro.index_basic(market='SZSE')
s1 = set(df['ts_code'].tolist())
s2 = set(res_sz)
print(len(s1),len(s2))
print(len(s1 & s2)) 
index_sz = list(s1 & s2)

459 65
65


In [33]:
df = pro.index_basic(market='CSI')
s1 = set(df['ts_code'].tolist())
s2 = set(res_csi)
print(len(s1),len(s2))
print(len(s1 & s2)) 
index_csi = list(s1 & s2)

3699 97
85


In [38]:
index_code = index_sh + index_sz + index_csi
len(index_code)
index_code

['000987.SH',
 '000971.SH',
 '000018.SH',
 '000906.SH',
 '000037.SH',
 '000029.SH',
 '000905.SH',
 '000823.SH',
 '000998.SH',
 '000982.SH',
 '000852.SH',
 '000913.SH',
 '000009.SH',
 '000065.SH',
 '000819.SH',
 '000064.SH',
 '000914.SH',
 '000841.SH',
 '000903.SH',
 '000056.SH',
 '000928.SH',
 '000001.SH',
 '000036.SH',
 '000808.SH',
 '000974.SH',
 '000993.SH',
 '000068.SH',
 '000805.SH',
 '000038.SH',
 '000827.SH',
 '000802.SH',
 '000933.SH',
 '000814.SH',
 '000688.SH',
 '000069.SH',
 '000300.SH',
 '000063.SH',
 '000901.SH',
 '000066.SH',
 '000046.SH',
 '000991.SH',
 '000935.SH',
 '000044.SH',
 '000021.SH',
 '000992.SH',
 '000015.SH',
 '000986.SH',
 '000858.SH',
 '000932.SH',
 '000048.SH',
 '000016.SH',
 '000984.SH',
 '000989.SH',
 '000934.SH',
 '000043.SH',
 '000010.SH',
 '000042.SH',
 '399997.SZ',
 '399610.SZ',
 '399348.SZ',
 '399707.SZ',
 '399992.SZ',
 '399966.SZ',
 '399006.SZ',
 '399986.SZ',
 '399998.SZ',
 '399991.SZ',
 '399974.SZ',
 '399368.SZ',
 '399295.SZ',
 '399987.SZ',
 '3993

In [39]:
df = pro.index_basic()
df.head()

,ts_code,name,market,publisher,category,base_date,base_point,list_date
0,000001.CZC,农期指数,OTH,郑州商品交易所,商品指数,None,1000.0,None
1,000001.SH,上证指数,SSE,中证公司,综合指数,19901219,100.0,19910715
2,000002.CZC,农期谷物,OTH,郑州商品交易所,商品指数,None,1000.0,None
3,000002.SH,上证A指,SSE,中证公司,综合指数,19901219,100.0,19920221
4,000003.CZC,农期软商,OTH,郑州商品交易所,商品指数,None,1000.0,None


In [43]:
df_t = df[df['ts_code'].isin(index_code)]

In [44]:
df_t.head()

,ts_code,name,market,publisher,category,base_date,base_point,list_date
1,000001.SH,上证指数,SSE,中证公司,综合指数,19901219,100.00,19910715
17,000009.SH,上证380,SSE,中证公司,规模指数,20031231,1000.00,20101129
19,000010.SH,上证180,SSE,中证公司,规模指数,20020628,3299.06,20020701
24,000015.SH,上证红利,SSE,中证公司,主题指数,20041231,1000.00,20050104
25,000016.SH,上证50,SSE,中证公司,规模指数,20031231,1000.00,20040102


In [48]:
len(df_t['ts_code'].tolist())

207

In [49]:
len(df_t['name'].tolist())

207

In [57]:
import json 

final_res = dict(zip(df_t['name'].tolist(),df_t['ts_code'].tolist())
                )
with open('D:\E-BOOK\\final_index_code.txt', 'w') as f:
    json_str = json.dumps(final_res,indent=0,ensure_ascii=False)
    f.write(json_str)
    f.write('\n')